In [1]:
#!pip install tldextract -q

# import library
import numpy as np
import pandas as pd
import re
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
#import plotly.express as px
#import plotly.io as pio
#from plotly.subplots import make_subplots
import seaborn as sns
import gc
import random
import os
import pickle
import tensorflow as tf
from tensorflow.python.util import deprecation
from urllib.parse import urlparse
import tldextract
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, layers, backend, metrics
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report





c:\users\apurva\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\apurva\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\apurva\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\apurva\

In [2]:
data = pd.read_csv('data.csv')

data1=data.head(100)
data2=data.tail(100)
data1.shape
data2.shape
frames = [data1,data2]
result = pd.concat(frames)
result.to_csv('my.csv', mode='a', header=False)
display(result)
data=result

url label
0                               diaryofagameaddict.com   bad
1                                     espdesign.com.au   bad
2                                   iamagameaddict.com   bad
3                                        kalantzis.net   bad
4                                slightlyoffcenter.net   bad
...                                                ...   ...
415882          en.wikipedia.org/wiki/Salisbury_School  good
415883            en.wikipedia.org/wiki/Sally_Fletcher  good
415884              en.wikipedia.org/wiki/Sally_Phipps  good
415885  en.wikipedia.org/wiki/Sally_Starr_(TV_hostess)  good
415886      en.wikipedia.org/wiki/Sally_of_the_Sawdust  good

[200 rows x 2 columns]

In [3]:
val_size = 0.2
train_data, val_data = train_test_split(data, test_size=val_size, stratify=data['label'], random_state=0)
fig = go.Figure([go.Pie(labels=['Train Size', 'Validation Size'], values=[train_data.shape[0], val_data.shape[0]])])
fig.update_layout(title='Train and Validation Size')
fig.show()

In [4]:
fig = go.Figure([go.Pie(labels=['Good', 'Bad'], values=data.label.value_counts())])
fig.update_layout(title='Percentage of Class (Good and Bad)')
fig.show()

In [5]:
def parsed_url(url):
    # extract subdomain, domain, and domain suffix from url
    # if item == '', fill with '<empty>'
    subdomain, domain, domain_suffix = ('<empty>' if extracted == '' else extracted for extracted in tldextract.extract(url))
    
    return [subdomain, domain, domain_suffix]

def extract_url(data):
    # parsed url
    extract_url_data = [parsed_url(url) for url in data['url']]
    extract_url_data = pd.DataFrame(extract_url_data, columns=['subdomain', 'domain', 'domain_suffix'])
    
    # concat extracted feature with main data
    data = data.reset_index(drop=True)
    data = pd.concat([data, extract_url_data], axis=1)
    
    return data

def get_frequent_group(data, n_group):
    # get the most frequent
    data = data.value_counts().reset_index(name='values')
    
    # scale log base 10
    data['values'] = np.log10(data['values'])
    
    # calculate total values
    # x_column (subdomain / domain / domain_suffix)
    x_column = data.columns[1]
    data['total_values'] = data[x_column].map(data.groupby(x_column)['values'].sum().to_dict())
    
    # get n_group data order by highest values
    data_group = data.sort_values('total_values', ascending=False).iloc[:, 1].unique()[:n_group]
    data = data[data.iloc[:, 1].isin(data_group)]
    data = data.sort_values('total_values', ascending=False)
    
    return data

def plot(data, n_group, title):
    data = get_frequent_group(data, n_group)
    fig = px.bar(data, x=data.columns[1], y='values', color='label')
    fig.update_layout(title=title)
    fig.show()

# extract url
data = extract_url(data)
train_data = extract_url(train_data)
val_data = extract_url(val_data)

In [6]:
fig = go.Figure([go.Bar(
    x=['domain', 'Subdomain', 'Domain Suffix'], 
    y = [data.domain.nunique(), data.subdomain.nunique(), data.domain_suffix.nunique()]
)])
fig.show()

In [7]:
import plotly.express as px
plot(
    data=data.groupby('label')['domain'], 
    n_group=20, 
    title='Top 20 Domains Grouped By Labels (Logarithmic Scale)'
)

In [8]:
data[(data['domain'] == 'google') & (data['label'] == 'bad')].head()

Empty DataFrame
Columns: [url, label, subdomain, domain, domain_suffix]
Index: []

In [9]:
plot(
    data=data.groupby('label')['subdomain'], 
    n_group=20, 
    title='Top 20 Sub Domains Grouped By Labels (Logarithmic Scale)'
)

In [10]:
X =data['url']
Y = data['label']
Y=Y.replace((np.inf, -np.inf, np.NaN), False).reset_index(drop=True)

Y.isnull().sum()
Y

0       bad
1       bad
2       bad
3       bad
4       bad
       ... 
195    good
196    good
197    good
198    good
199    good
Name: label, Length: 200, dtype: object

In [11]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
Y= label_encoder.fit_transform(Y)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30%

In [12]:
#import pickle

from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300)

clf.fit(X, Y)


MLPClassifier(max_iter=300, random_state=1)

In [13]:

yp=clf.predict(X)
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y, yp))

Accuracy: 1.0


In [14]:
print("Precision:",metrics.precision_score(Y, yp))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(Y, yp))

Precision: 1.0
Recall: 1.0


In [15]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)

clf.fit(X, Y)

yp=clf.predict(X)
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y, yp))
print("Precision:",metrics.precision_score(Y, yp))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(Y, yp))


Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [16]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=7)

clf.fit(X, Y)

yp=clf.predict(X)
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y, yp))
print("Precision:",metrics.precision_score(Y, yp))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(Y, yp))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [ ]:
import pymysql as mdb
import time
def upd(res,ids):
    
    mydb = mdb.connect(
      host="127.0.0.1",
      user="root",
      passwd="",
      database="botnet"
    )

    mycursor = mydb.cursor()

   

    sql = "UPDATE url_data SET result = %s WHERE room_id = %s"
    val = (res,ids)

    mycursor.execute(sql, val)

    mydb.commit()
    mydb.close()
while True:
    time.sleep(2)
    mydb = mdb.connect(
          host="127.0.0.1",
          user="root",
          passwd="",
          database="botnet"
        )

    mycursor = mydb.cursor()

    sql = "SELECT details,result,room_id FROM url_data"


    mycursor.execute(sql)
   
    myresult = mycursor.fetchall()
    mydb.close()
    for x in myresult:            
        comm=str(x[0])
        chu=str(x[1])
        ids=str(x[2])
        
        if(chu==' '):
            
            a=[]
            comm=str(comm)
            a.append(comm)
            ne=cv.transform(a)
            pred=clf.predict(ne)
            res=str(pred[0])
            print(res)
            if res=='0':
                res='Malicious Bot'
            else:
                res='Legitimate Bot'

            upd(res,ids)

0
1
1
0
1
